# **Meridian Scenario Planner Beta Demo**

<style>
devsite-code .tfo-notebook-code-cell-output {
  max-height: 300px;
  overflow: auto;
  background: rgba(255, 247, 237, 1);  /* orange bg to distinguish from input code cells */
}

devsite-code .tfo-notebook-code-cell-output + .devsite-code-buttons-container button {
  background: rgba(255, 247, 237, .7);  /* orange bg to distinguish from input code cells */
}

devsite-code[dark-code] .tfo-notebook-code-cell-output {
  background: rgba(64, 78, 103, 1);  /* medium slate */
}

devsite-code[dark-code] .tfo-notebook-code-cell-output + .devsite-code-buttons-container button {
  background: rgba(64, 78, 103, .7);  /* medium slate */
}

/* override default table styles for notebook buttons */
.devsite-table-wrapper .tfo-notebook-buttons {
  display: inline-block;
  margin-left: 3px;
  width: auto;
}

.tfo-notebook-buttons tr {
  background: 0;
  border: 0;
}

.tfo-notebook-buttons td {
  padding-left: 0;
  padding-right: 20px;
}

.tfo-notebook-buttons {
  --tfo-notebook-buttons-box-shadow: 0 1px 2px 0 rgba(60, 64, 67, .3), 0 1px 3px 1px rgba(60, 64, 67, .15);
}

.tfo-notebook-buttons a,
.tfo-notebook-buttons :link,
.tfo-notebook-buttons :visited {
  border-radius: 8px;
  box-shadow: var(--tfo-notebook-buttons-box-shadow);
  color: #202124;
  padding: 12px 24px;
  transition: box-shadow 0.2s;
}

.tfo-notebook-buttons a:hover,
.tfo-notebook-buttons a:focus {
  box-shadow: var(--tfo-notebook-buttons-box-shadow);
}

.tfo-notebook-buttons td > a {
  -webkit-box-align: center;
  -ms-flex-align: center;
  align-items: center;
  display: -webkit-box;
  display: -ms-flexbox;
  display: flex;
}

.tfo-notebook-buttons td > a > img {
  margin-right: 8px;
}
</style>

<table class="tfo-notebook-buttons tfo-api nocontent" align="left">
  <tbody>
    <tr>
      <td>
        <a target="_blank" href="https://github.com/google/meridian/blob/main/demo/Meridian_Scenario_Planner_Beta.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
      </td>
    </tr>
  </tbody>
</table>

Welcome to the Meridian Scenario Planner Beta Demo. This colab will generatate an interactive Looker Studio dashboard based on a trained model.

If you are new to Looker Studio or Meridian, it will be helpful to review the following resources:
<ul>
 <li><a href="https://developers.google.com/meridian/docs/scenario-planning/meridian-scenario-planner">Meridian Scenario Planner Beta User Guide</a></li>
  <li><a href="https://developers.google.com/meridian/notebook/meridian-getting-started">Get started with Meridian</a></li>
  <li><a href="https://cloud.google.com/looker/docs/studio/quick-start-guide">Looker Studio</a></li>
</ul>

To get started with Meridian Scenario Planner, follow the detailed instructions in this Colab, which covers the following entries:

<ol start="0">
  <li><a href="#install">Install and setup</a></li>
  <li><a href="#load-model">Train a simple model</a></li>
  <li><a href="#dashboard">Generate dashboard</a></li>
  <li><a href="#save-and-sharing">Save and share dashboard</a></li>
</ol>

Please use <a href="https://docs.google.com/forms/d/1LBx5S1fdVP5IBt6RHfhj38Pepkik7jzI3KtI4udviqg"> user feedback form </a> to share feedback with Meridian team.

<a name="install"></a>
## Install and setup

1\. Make sure you are using one of the available GPU Colab runtimes which is **required** to run Meridian optimization. You can change your notebook's runtime in `Runtime > Change runtime type` in the menu. All users can use the T4 GPU runtime which is sufficient to run the demo colab, free of charge. Users who have purchased one of Colab's paid plans have access to premium GPUs (such as V100, A100 or L4 Nvidia GPU).

2\. Run the cell below to install necessary components and setup the environment. This step will not use GPU and it shoud take 3-5 mins to complete.

In [ ]:
# @markdown Then we can install meridian library with scenario planner feature, this step can take 3-7 mins
!pip install google-meridian[scenarioplanner,and-cuda]


In [ ]:
# @markdown Now, run this cell and we can test if all dependencies are appropratly installed
import tensorflow_probability as tfp
import pandas as pd

from meridian import constants
from meridian.data import data_frame_input_data_builder
from meridian.model import model
from meridian.model import prior_distribution
from meridian.model import spec

from schema.processors import model_fit_processor
from schema.processors import marketing_processor
from schema.processors import budget_optimization_processor
from schema.utils import date_range_bucketing

from scenarioplanner.converters import sheets
from scenarioplanner.converters.dataframe import dataframe_model_converter
from scenarioplanner import mmm_ui_proto_generator as mmm_ui_gen
from scenarioplanner.linkingapi import url_generator

<a name="load-model"></a>
## Get a model (choose one cell to run)

In [ ]:
# @markdown [Train New Model] If you would like to train a simple model with [Demo data](https://raw.githubusercontent.com/google/meridian/refs/heads/main/meridian/data/simulated_data/csv/geo_all_channels.csv),  run this cell. This will take about 10-20 mins using GPU.

df = pd.read_csv(
    "https://raw.githubusercontent.com/google/meridian/refs/heads/main/meridian/data/simulated_data/csv/geo_all_channels.csv"
)

media_channels = ["Channel0", "Channel1", "Channel2", "Channel3", "Channel4"]
non_media_channels = ["Promo"]
organic_channels = ["Organic_channel0"]
builder = (
    data_frame_input_data_builder.DataFrameInputDataBuilder(
    kpi_type='non_revenue')
    .with_kpi(df, kpi_col="conversions")
    .with_revenue_per_kpi(df, revenue_per_kpi_col="revenue_per_conversion")
    .with_population(df)
    .with_controls(
        df, control_cols=["sentiment_score_control", "competitor_sales_control"])
    .with_media(
        df,
        media_cols=[f"{channel}_impression" for channel in media_channels],
        media_spend_cols=[f"{channel}_spend" for channel in media_channels],
        media_channels=media_channels)
    .with_non_media_treatments(df,non_media_treatment_cols=non_media_channels)
    .with_organic_media(
        df,
        organic_media_cols=[f"{channel}_impression" for channel in organic_channels],
        organic_media_channels=organic_channels,
        media_time_col=constants.TIME)
)

data = builder.build()


roi_mu = 0.2  # Mu for ROI prior for each media channel.
roi_sigma = 0.9  # Sigma for ROI prior for each media channel.
prior = prior_distribution.PriorDistribution(
    roi_m=tfp.distributions.LogNormal(roi_mu, roi_sigma, name=constants.ROI_M)
)
model_spec = spec.ModelSpec(prior=prior)

meridian = model.Meridian(input_data=data, model_spec=model_spec)
meridian.sample_prior(500)
meridian.sample_posterior(
    n_chains=10, n_adapt=2000, n_burnin=500, n_keep=1000, seed=1
)

In [ ]:


# @markdown [Google Drive] If your trained model is in Google Drive, run this cell with your `model_path`, otherwise, open this cell and change the configuration.
from google.colab import drive
drive_mount = '/content/drive'
drive.mount(drive_mount)

# @markdown Provide complete path to the pickle file(For mounted Google Drive, a trained model is stored in `MyDrive/<Path to pkl file>`):
model_path= ''# @param {"type":"string","placeholder": "Drive path of your saved model"}
meridian = model.load_mmm(f'{drive_mount}/{model_path}')
meridian = model.Meridian(input_data=meridian.input_data, model_spec= meridian.model_spec, inference_data=meridian.inference_data)

# @markdown Runing this cell will load the model from the provided path

In [ ]:
import os
from google.colab import auth

auth.authenticate_user()
# @markdown [Google Cloud Storage] If you are an Enterprise user and would like to load model from GCS, run this cell with your bucket name and path
project_id = ""# @param {"type":"string","placeholder": "Cloud project id"}
bucket_name = "" # @param {"type":"string","placeholder": "GCS bucket that contains your model"}
model_path = "" # @param {"type":"string","placeholder": "<folders>/<mdoel_name>.pkl"}
os.environ['GOOGLE_CLOUD_PROJECT'] = project_id
!gcloud config set project {project_id}
!mkdir /content/{bucket_name}

# Uncomment below if you don't have gcsfuse installed
!echo "deb https://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!apt-get update
!apt-get install gcsfuse

!gcsfuse --implicit-dirs {bucket_name} /content/{bucket_name}
meridian = model.load_mmm(f'/content/{bucket_name}/{model_path}')
meridian = model.Meridian(input_data=meridian.input_data, model_spec= meridian.model_spec, inference_data=meridian.inference_data)
!fusermount -u /content/{bucket_name}

In [ ]:
# @markdown [Third Party storage] If the model lives in a third party storage(SFTP, SSH ), please download the model to local and run this cell with `local_model_path`

local_model_path = "" # @param {"type":"string","placeholder": "/content/....pkl"}
meridian = model.load_mmm(local_model_path)

<a name="dashboard"></a>
## Generate dashboard

Similar to the two pager generated by the Meridian library, we need to push the model inference results to Google Sheet so that Looker Studio can access the it.

This step needs Google Drive access and Google Sheet read and write access, please grant all access as requested in the pop-up window.

Now, let's configure how we would like the sheet to be generated. This step can take long time(< 20 mins) in a GPU enabled session, since it will do multiple inferencing based on provided configuration below.

In [ ]:
# @markdown Grant Colab access to create Google Sheet in your Drive
from google.colab import auth

_SCOPES = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive",
]
credentials = auth.authenticate_user(_SCOPES)

In [ ]:
mmm=meridian

# @title Define specs for model analyses and budget optimizations

# @markdown ## General setup
spreadsheet_name = "Meridian Looker Studio Small Demo Data"  # @param {"type":"string"}
optimization_name = "Colab demo"  # @param {"type":"string"}
include_non_paid_channels = True  # @param {"type":"boolean","placeholder":"Whether to include non-paid channels in the analysis."}

start_date = None
end_date = None


# @markdown ## Time breakdown setup
# @markdown By default, the analysis results and budget scenarios are based on
# @markdown the whole time period in the dataset. The following enables you to
# @markdown generate additional results for more granular time frames.

# @markdown Breakdowns:
yearly = False  # @param {"type":"boolean","placeholder":"Whether to optimize budget for every year"}
quarterly = True  # @param {"type":"boolean","placeholder":"Whether to optimize budget for every quarter"}
monthly = False  # @param {"type":"boolean","placeholder":"Whether to optimize budget for every month"}

# @markdown ## Interactive budget optimization setup
# @markdown The following controls the size of the optimization grid. The larger
# @markdown the grid, the more adjustment can be done during the interactive
# @markdown budget optimization when using the Meridian Looker Studio report.

# @markdown * `min_spend_shift_ratio` determines the minimum available spend by
# @markdown reducing the original spend by the specified proportion.

# @markdown * `max_spend_shift_ratio` determines the maximum available spend by
# @markdown increasing the original spend by the specified proportion.
min_spend_shift_ratio = 1.0  # @param {"type":"raw","placeholder":"From 0 to 1."}
max_spend_shift_ratio = 1.0  # @param {"type":"raw","placeholder":"A number greater than 0."}


# @markdown If the input data contains reach and frequency channels, you can choose to use optimal frequency during budget optimization:
use_optimal_frequency = True # @param {"type":"boolean","placeholder":"Whether to optimize budget with optimal frequency data"}
# @markdown If the frequency range is too wide, this cell would take longer time to finish, please consider limit the max frequency if you experienced a timeout issue:
max_frequency = 10.0 # @param {"type":"raw","placeholder":"A number greater than 0."}
# @markdown If the max frequency from the input data is more than 30, please consider limit the max frequency to 10 or less.

time_breakdown_generators = []

if yearly:
  time_breakdown_generators.append(
      date_range_bucketing.YearlyDateRangeGenerator
  )

if quarterly:
  time_breakdown_generators.append(
      date_range_bucketing.QuarterlyDateRangeGenerator
  )

if monthly:
  time_breakdown_generators.append(
      date_range_bucketing.MonthlyDateRangeGenerator
  )

channel_constraints = []

if min_spend_shift_ratio is not None and max_spend_shift_ratio is not None:
  for channel in mmm.input_data.get_all_paid_channels():
    channel_constraints.append(
        budget_optimization_processor.ChannelConstraintRel(
            channel_name=channel,
            spend_constraint_lower=min_spend_shift_ratio,
            spend_constraint_upper=max_spend_shift_ratio,
        )
    )

grid_name_prefix = ("-").join(optimization_name.lower().split(" "))

budgetOptSpec = budget_optimization_processor.BudgetOptimizationSpec(
    start_date=start_date,
    end_date=end_date,
    optimization_name=optimization_name,
    grid_name=grid_name_prefix,
    constraints=channel_constraints,
    use_optimal_frequency=use_optimal_frequency,
    max_frequency=max_frequency,
)
print("Creating the proto data.")
summary_spec = marketing_processor.MediaSummarySpec(
    include_non_paid_channels=include_non_paid_channels)
mmm_proto = mmm_ui_gen.create_mmm_ui_data_proto(
    mmm=mmm,
    specs=[
        model_fit_processor.ModelFitSpec(),
        marketing_processor.MarketingAnalysisSpec(
            media_summary_spec=summary_spec,
        ),
        budgetOptSpec,
    ],
    time_breakdown_generators=time_breakdown_generators,
)
print("Converting the proto to a dataframe")
converter = dataframe_model_converter.DataFrameModelConverter(mmm_proto)
dataframes = converter()
print("Uploading data to a Google spreadsheet.")
if spreadsheet_name:
  spreadsheet = sheets.upload_to_gsheet(
      dataframes, credentials, spreadsheet_name=spreadsheet_name
  )
else:
  spreadsheet = sheets.upload_to_gsheet(dataframes, credentials)
print(f"Spreadsheet URL: {spreadsheet.url}")

In [ ]:
# @markdown Run this cell to access the Meridian Scenario Planner in Looker Studio
url = url_generator.create_report_url(spreadsheet)
html_link = f'<a href="{url}">Click to create a report</a>'
from IPython.display import HTML
HTML(html_link)

<a name="save-and-share"></a>
## Save and share the dashboard

When you click the link above, the dashboard is not sharable and the visualization might not be visible yet, since the dashbaord is not saved anywhere. Lets' do the following:

1. Hover on top and click on the Edit and save button1. Click on Ackonowledge and save1. Click on Allow to enable community visulizations, you only need to do it once1. The community visualization might still be disabled:
1. Refresh the page, the dashboard should display correctly.

Now, the dashboard should display correctly, and it should be sharable by click on the Share button on top. Please refer to <a href="https://developers.google.com/meridian/docs/scenario-planning/meridian-scenario-planner">Meridian Scenario Planner User Guide</a> on the further usage of the dashboard

Thanks for using Meridian Scenario Planner, your feedback is valuable to us, please share your feedback through [user feedback form](https://docs.google.com/forms/d/1LBx5S1fdVP5IBt6RHfhj38Pepkik7jzI3KtI4udviqg).